# Simulate Datasets for Evaluation

Welcome! This notebook will walk you through generating synthetic datasets using Azure AI Search and the Azure AI Evaluation Simulator.

## What You'll Learn
- How to configure the Azure AI Evaluation Simulator
- How to connect to Azure AI Search to retrieve content
- How to create a RAG application callback
- How to generate synthetic query-response pairs
- How to save and review the generated dataset

Let's get started! 🚀

---

## Step 1: Verify Environment Variables

The following environment variables should already be configured in your `.env` file from the earlier setup steps:

- **AZURE_OPENAI_API_KEY**: Your Azure OpenAI API key
- **AZURE_OPENAI_ENDPOINT**: Your Azure OpenAI service endpoint
- **AZURE_OPENAI_API_VERSION**: The API version to use
- **AZURE_OPENAI_DEPLOYMENT**: The name of your deployed chat model
- **AZURE_SEARCH_ENDPOINT**: Your Azure AI Search service endpoint
- **AZURE_SEARCH_API_KEY**: Your Azure AI Search API key
- **AZURE_SEARCH_INDEX_NAME**: The name of your search index

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify all required Azure service credentials are available
required_vars = [
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_ENDPOINT", 
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_DEPLOYMENT",
    "AZURE_SEARCH_ENDPOINT",
    "AZURE_SEARCH_API_KEY",
    "AZURE_SEARCH_INDEX_NAME"
]

missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    raise EnvironmentError(f"❌ Missing environment variables: {', '.join(missing_vars)}")

print("✅ All environment variables configured!")

✅ All environment variables configured!


In [2]:
# Initialize Azure AI Search connection parameters
search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
search_api_key = os.environ.get("AZURE_SEARCH_API_KEY") 
search_index_name = os.environ.get("AZURE_SEARCH_INDEX_NAME")

print(f"✅ Azure AI Search configured with index: {search_index_name}")

✅ Azure AI Search configured with index: zava-products


## Step 2: Configure Azure OpenAI Model

Now let's create the model configuration that the simulator will use to generate synthetic queries and responses.

In [3]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

# Create model configuration using environment variables
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"), 
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

print(f"✅ Model configuration created for: {model_config['azure_deployment']}")

✅ Model configuration created for: gpt-4.1


## Step 3: Initialize the Simulator

Let's create the Azure AI Evaluation Simulator instance using the model configuration.

In [4]:
# 4. Initialize the Azure AI Evaluation Simulator
from azure.ai.evaluation.simulator import Simulator

# Create simulator instance with the configured model
simulator = Simulator(model_config=model_config)
print("📊 Simulator initialized successfully!")

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


📊 Simulator initialized successfully!


## Step 4: Define Search Function

This function will search the Azure AI Search index for relevant content based on queries. It will be used by the RAG application callback to retrieve context.

In [5]:
# 5. Define function to search Azure AI Search index and retrieve content
import requests
import json

def search_index_for_content(query: str, top_k: int = 5) -> str:
    """
    Search the Azure AI Search index for relevant content based on a query.
    
    Args:
        query (str): Search query to find relevant content
        top_k (int): Number of top results to retrieve (default: 5)
    
    Returns:
        str: Combined content from search results, truncated to 1000 characters
    """
    # Construct the search API endpoint
    search_url = f"{search_endpoint}/indexes/{search_index_name}/docs/search?api-version=2023-11-01"
    
    # Set up request headers with API key authentication
    headers = {
        "Content-Type": "application/json",
        "api-key": search_api_key
    }
    
    # Define the search query payload
    search_payload = {
        "search": query,
        "top": top_k,
        "select": "content,title"  # Updated to match available fields
    }
    
    try:
        # Execute the search request
        response = requests.post(url=search_url, headers=headers, json=search_payload)
        response.raise_for_status()  # Raise an exception for HTTP error codes
        
        # Parse the search results
        search_results = response.json()
        combined_content = ""
        
        # Extract and combine content from search results
        for result in search_results.get("value", []):
            # Prioritize 'content' field, fall back to 'title'
            content = result.get("content") or result.get("title", "")
            if content:
                combined_content += content + " "
        
        # Limit content length to prevent token limits and improve performance
        return combined_content[:1000].strip()
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Error searching index: {e}")
        return f"Error retrieving content for query: {query}"
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing search response: {e}")
        return f"Error processing search results for query: {query}"

## Step 5: Test the Search Function

Let's verify that the search function works correctly by testing it with a sample query.

In [6]:
# Test the search function with a sample query
test_query = "eggshell paint"
retrieved_content = search_index_for_content(test_query)

print(f"✅ Search test successful!")
print(f"📄 Retrieved response: {retrieved_content} for query: '{test_query}'")

✅ Search test successful!
📄 Retrieved response: Durable eggshell finish paint with subtle sheen, ideal for living rooms and bedrooms with easy cleanup. Disposable plastic liners that fit standard paint trays for quick cleanup and color changes. Environmentally friendly zero-VOC paint for healthy indoor air quality in all living spaces. Premium acrylic exterior paint with fade resistance and superior adhesion for long-lasting protection. Washable semi-gloss interior paint for kitchens, bathrooms, and trim work with moisture resistance. for query: 'eggshell paint'


## Step 6: Create RAG Application Callback

This callback function simulates a RAG (Retrieval-Augmented Generation) application that:
- Extracts user queries from messages
- Searches the index for relevant content
- Generates responses using Azure OpenAI
- Returns formatted responses for the simulator

In [7]:
# 7. Define the application callback function for the simulator
from typing import Dict, Any, Optional
from openai import AzureOpenAI

async def rag_application_callback(
    messages: Dict,
    stream: bool = False,
    session_state: Any = None,
    context: Optional[Dict[str, Any]] = None,
) -> Dict:
    """
    Callback function that simulates a RAG (Retrieval-Augmented Generation) application.
    
    This function:
    1. Extracts the user query from the message
    2. Searches the Azure AI Search index for relevant content
    3. Uses Azure OpenAI to generate a response based on the retrieved content
    4. Returns the response in the expected format
    
    Args:
        messages (Dict): Message history containing user queries
        stream (bool): Whether to stream the response (not used in this implementation)
        session_state (Any): Session state information
        context (Optional[Dict[str, Any]]): Additional context information
    
    Returns:
        Dict: Response containing the generated message and metadata
    """
    # Extract the user's query from the latest message
    messages_list = messages["messages"]
    user_query = messages_list[-1]["content"]
    
    # Initialize Azure OpenAI client
    openai_client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
    
    # Retrieve relevant content from the search index
    retrieved_context = search_index_for_content(user_query)
    
    # Create a system prompt that instructs the model to use the retrieved context
    system_prompt = """You are a polite and helpful assistant that answers questions based on the provided context. 
Use the context information to provide accurate and relevant responses. If the context doesn't contain 
enough information to answer the question, say so politely. If the context mentions a product by name, reference it in the response."""
    
    # Generate response using Azure OpenAI
    try:
        completion = openai_client.chat.completions.create(
            model=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Context: {retrieved_context}"},
                {"role": "user", "content": f"Question: {user_query}"}
            ],
            max_tokens=500,
            temperature=0.7,
        )
        
        # Extract the generated response
        ai_response = completion.choices[0].message.content
        
    except Exception as e:
        ai_response = f"Sorry, I encountered an error while generating a response: {str(e)}"
    
    # Format the response according to the expected structure
    response_message = {
        "content": ai_response,
        "role": "assistant",
        "context": retrieved_context,
    }
    
    # Add the response to the message history
    messages["messages"].append(response_message)
    
    # Return the complete response structure
    return {
        "messages": messages["messages"], 
        "stream": stream, 
        "session_state": session_state, 
        "context": retrieved_context
    }

## Step 7: Generate Synthetic Dataset

Now we'll use the simulator to generate synthetic query-response pairs based on content from the search index. This will create 5 query-response pairs for evaluation.

In [8]:
# Verify the retrieved content is ready for simulation
if not retrieved_content:
    raise ValueError("No content retrieved from search. Please run Step 5 again with a valid query.")

print(f"✅ Ready to simulate with {len(retrieved_content)} characters of content")

✅ Ready to simulate with 492 characters of content


In [10]:
from pathlib import Path

# Run the simulator to generate synthetic data
# The simulator needs a list of query-response dictionaries or plain text
synthetic_outputs = await simulator(
    target=rag_application_callback,
    text=retrieved_content,
    num_queries=10,
    max_conversation_turns=1,
)

print(f"✅ Generated {len(synthetic_outputs)} synthetic query-response pairs!")

/home/vscode/.local/lib/python3.12/site-packages/azure/ai/evaluation/simulator/_simulator.py:157: UserWarning: You have specified 'num_queries' > len('tasks') (10 > 0). All tasks will be used for generation and the remaining 10 lines will be simulated in task-free mode
  warnings.warn(
Generating: 100%|██████████████████████████████████████████████| 10/10 [00:23<00:00,  2.31s/message]

✅ Generated 10 synthetic query-response pairs!


## Step 8: Save the Dataset to File

Let's save the generated dataset to a JSONL file for use in evaluation workflows.

In [11]:
output_file = Path("21-simulate-datasets.jsonl")

# Write each output as a JSON line to the file
with output_file.open("w") as f:
    for output in synthetic_outputs:
        f.write(output.to_eval_qr_json_lines())

print(f"✅ Dataset saved to: {output_file.absolute()}")

✅ Dataset saved to: /workspaces/ignite25-PDY123-learn-how-to-observe-manage-and-scale-agentic-ai-apps-using-azure/labs/2-models/21-simulate-datasets.jsonl


## Step 9: Preview the Dataset

Let's load and display a preview of the generated dataset to verify its structure and quality.

In [13]:
import pandas as pd

# Load and display the dataset
dataset_df = pd.read_json(output_file, lines=True)

print(f"✅ Dataset loaded: {dataset_df.shape[0]} rows, {dataset_df.shape[1]} columns")
print(f"📋 Columns: {list(dataset_df.columns)}")
print()

# Show first few records
dataset_df.head(10)

✅ Dataset loaded: 10 rows, 3 columns
📋 Columns: ['query', 'response', 'context']



,query,response,context
0,What type of finish does the durable eggshell ...,The durable eggshell finish paint has a subtle...,Durable eggshell finish paint with subtle shee...
1,Which rooms are recommended for the durable eg...,The durable eggshell finish paint with subtle ...,Durable eggshell finish paint with subtle shee...
2,What feature makes the durable eggshell finish...,The durable eggshell finish paint is easy to m...,Durable eggshell finish paint with subtle shee...
3,What product fits standard paint trays for qui...,The product that fits standard paint trays for...,Disposable plastic liners that fit standard pa...
4,What is the main environmental benefit of the ...,The main environmental benefit of the zero-VOC...,Environmentally friendly zero-VOC paint for he...
5,What type of paint is recommended for all livi...,The environmentally friendly zero-VOC paint is...,Environmentally friendly zero-VOC paint for he...
6,Which paint is described as premium acrylic ex...,The paint described as premium acrylic exterio...,Premium acrylic exterior paint with fade resis...
7,What are the two main features of the premium ...,The two main features of the premium acrylic e...,Premium acrylic exterior paint with fade resis...
8,"Which paint is recommended for kitchens, bathr...",The washable semi-gloss interior paint is reco...,Washable semi-gloss interior paint for kitchen...
9,What is the key feature of the washable semi-g...,The key feature of the washable semi-gloss int...,Washable semi-gloss interior paint for kitchen...


## Step 10: Review the Dataset File

Open the `21-simulate-datasets.jsonl` file in VS Code to examine the generated query-response-context triples and verify the data quality.

## Next Steps

You've successfully generated a synthetic evaluation dataset! You can now use this dataset to:
- Evaluate your RAG application's retrieval quality
- Test and fine-tune your system prompts
- Benchmark your application's performance
- Create test cases for development

Great work! 🎉